## Transform Customer Data
1. Remove records with NULL customer_id 
1. Remove exact duplicate records
1. Remove duplicate records based on created_timestamp
1. CAST the columns to the correct Data Type
1. Write transformed data to the Silver schema

In [0]:
df = spark.sql('''
               SELECT * 
               FROM gizmobox.bronze.py_customers
               ''')
display(df)

In [0]:
df = spark.table('gizmobox.bronze.py_customers')
display(df)

In [0]:
df = spark.read.table('gizmobox.bronze.py_customers')
display(df)

In [0]:
# 1.

# df_not_null = df.filter("customer_id IS NOT NULL")
#display(df_not_null)

# df_not_null = df.filter(df.customer_id.isNotNull())
# display(df_not_null)

df_not_null = df.where(df.customer_id.isNotNull())
display(df_not_null)


In [0]:
# 2.

# df_distinct = df_not_null.distinct()
# display(df_distinct)

df_distinct = df_not_null.dropDuplicates()
display(df_distinct)

In [0]:
from pyspark.sql import functions as F
df_max_ts = df_distinct.groupBy('customer_id') \
                        .agg(F.max('created_timestamp').alias('max_created_timestamp'))



In [0]:
display(df_max_ts)

In [0]:
# 3.
df_distinct_customer = (
    df_distinct
    .join(df_max_ts, (df_distinct.customer_id == df_max_ts.customer_id) &
                         (df_distinct.created_timestamp == df_max_ts.max_created_timestamp),
                         'inner')
    .select(df_distinct['*'])
)

In [0]:
display(df_distinct_customer)

In [0]:
# 4.

df_casted_customer = (
                    df_distinct_customer
                    .select(df_distinct_customer.created_timestamp.cast('timestamp'),
                            df_distinct_customer.customer_id,
                            df_distinct_customer.customer_name,
                            df_distinct_customer.date_of_birth.cast('date'),
                            df_distinct_customer.email,
                            df_distinct_customer.member_since.cast('date'),
                            df_distinct_customer.telephone)
)
display(df_casted_customer)

In [0]:
# 5. 
df_casted_customer.writeTo('gizmobox.silver.py_customers').createOrReplace()